## Here I translate GCN in tensorflow to GCN in tensorflow.keras
### This is the current method i'm focusing on 

In [1]:
# dependencies
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers

In [2]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.0-rc2
2.2.4-tf


In [3]:
# methods are functions associated with a class
# the data are the attributes

In [4]:
adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype(float) #takes the first 10,000 molecules

This class allows us to build the weight and biases for each convolutional layer and also each model. After building the biases in the build method, I will perform the matrix multiplications in the call method for each model. The cell below will iteratively call upon all layers, until the whole model is build. The model will then be compiled. 

In [5]:
from tensorflow.keras import layers
import conv_models

class MyLayer(layers.Layer):
    models = ['gcn', 'gcn_a', 'gcn_g', 'gcn_ag']
    
    def __init__(self, output_dim, adjacency, features, 
                 weight_shape, bias_shape, model, **kwargs):
        self.output_dim = output_dim
        self.features = tf.placeholder(shape = [100,50,58], 
                                       dtype = 'float64')
        self.adjacency = tf.placeholder(shape = [100,50,50], 
                                        dtype = 'float64')
        self.weight_shape = weight_shape
        self.bias_shape = bias_shape
        self.model = model
        self.kernel = None
        self.bias = None
        super(MyLayer, self).__init__(**kwargs)
    
    # still have to add attention weights
    # initializers may be incorrect
    def build(self):
        #self.weight_shape = weight_shape
        #self.bias_shape = bias_shape
        self.kernel = self.add_weight(name='kernel',
                                  shape=self.weight_shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype='float64')
        
        self.bias = self.add_weight(name='bias',
                                shape=self.bias_shape,
                                initializer='glorot_uniform',
                                trainable=True,dtype='float64')
        print(self.bias,self.kernel)
        
        #why did I have to get rid of this line?
        #super(MyLayer, self).build()
    
    #here is where I will implement all models, which will be built in another script. 
    def call(self):
        if self.model == MyLayer.models[0]:
            _features = conv_models.GCN(self.features,self.adjacency,
                                       self.kernel,self.bias)
        elif self.model == MyLayer.models[1]:
            print('go to gcn+att function and calc. mult.')
        elif self.model == MyLayer.models[2]:
            print('go to gcn+gate function and calc. mult.')
        elif self.model == MyLayer.models[3]:
            print('go to aug. gcn function and calc. mult.')
        else:
            raise ValueError('not a valid model')
        return(_features)
#look into the following functions, especially the last

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [6]:
#this is where all models will be built

#for now i've constructed the input layer whose output shape matches the output shape of Ryu et al's input layer

model = tf.keras.Sequential()
num_atoms = int(features.shape[1])
input_dim = int(features.shape[2])
hidden_dim = [input_dim]
num_layers=3

input_layer = MyLayer(32,adj,features,[input_dim,32],[32],'gcn')
input_layer.build() # get weights and bias
input_layer.call() # propogation rule argument
# now we need to relu, and repeat for L hidden layer times.

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'bias:0' shape=(32,) dtype=float64> <tf.Variable 'kernel:0' shape=(58, 32) dtype=float64>
Instructions for updating:
Use keras.layers.dense instead.
the shape Tensor("Relu:0", shape=(100, 50, 32), dtype=float64)


<tf.Tensor 'Relu:0' shape=(100, 50, 32) dtype=float64>